In [10]:
# This is a program to calculate the Faraday Polarization Rotation. We will focuse o the Verdet Constant of Rb vapor.
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
from sympy.physics.wigner import wigner_3j


def atomic_constant(Atom, p, n, start):
    '''
    Atom: Character string designating the atomic symbol
    P: Real Linear array to store atomic constants upon return
    N: Integer equal to the index of the isotope to be used
    start: Integer equal to 1 if the first time the function is called
    '''
    # Create a 12x8 zero matrix
    C = np.zeros((12, 8))

    # Fill columns with atomic data for 'LI006', 'LI007', 'NA023', 'K0039', 'K0041', 'RB085', 'RB087', 'CS133'
    # [upper level energy(cm-1), (cm-1), ap(cm-1), ac(cm-1), bp(cm-1), lower level energy(cm-1) ]
    C[0:12, 0] = [14903.89, 0.223559, 1.6918e-4, -1.2882e-4, -3.6860e-7, 0.0, 0.0050747387200, 0.8220300, 1.0, 26.4, 26.4,
         0.0742]
    C[0:12, 1] = [14903.89, 0.22355900, 4.4676e-4, -3.4017e-4, -1.8430e-5, 0.0, 0.0134010049, 3.2563600, 1.5, 26.4, 26.4,
         0.9258]
    C[0:12, 2] = [16967.647, 11.464, 1.177e-3, -5.837e-6, 2.35e-4, 0.0, 0.0295475433, 2.2174000, 1.5, 16.0, 16.0, 1.0]
    C[0:12, 3] = [13023.65, 38.480, 3.642e-4, 8.8950e-6, 2.310e-4, 0.0, 0.00770065604, 0.3914300, 1.5, 26.0, 26.0, 0.9312]
    C[0:12, 4] = [13023.6500, 38.48000, 2.004e-4, 4.894e-6, 2.820e-4, 0.0, 0.00423649534, 0.21487, 1.5, 26.0, 26.0, 0.0688]
    C[0:12, 5] = [12737.36, 158.4000, 1.525e-3, 2.496e-5, 2.0864e-3, 0.0, 0.0337537115, 1.3524, 2.5, 25.5, 28.1, 0.7215]
    C[0:12, 6] = [12737.36, 158.40, 5.149e-3, 8.428e-5, 1.0432e-3, 0.0, 0.113990236, 2.7500, 1.5, 25.5, 28.1, 0.2785]
    C[0:12, 7] = [11547.65, 369.41, 3.5692e-3, -2.254e-4, -3.2e-5, 0.0, 0.0766582975, 2.5779, 3.5, 29.9, 34.0, 1.0]

    FT = 1.499e9
    Label = ['LI006', 'LI007', 'NA023', 'K039', 'K041', 'RB085', 'RB087', 'CS133']

    # Find the correct isotope based on Atom string
    isotope_found = False
    for i in range(8):
        if Atom == Label[i][:2]:
            isotope_found = True
            if n == 0:
                k = i  # Select first isotope of the atom
            else:
                k = min(i + n, 7)  # For subsequent isotopes
            break

    if not isotope_found:
        print(f"Atom {Atom} not found in the Label list.")
        return p

    # Fill the array `p` with constants from matrix `C`
    for i in range(11):
        p[i] = C[i, k]

    # Perform computations on `p` array
    p[12] = np.sqrt(FT / (p[0] - p[1]) ** 3 / p[10])
    p[13] = np.sqrt(2.0 * FT / (p[0] + 0.5 * p[1]) ** 3 / p[9])
    p[14] = FT / (p[0] - p[1]) ** 2 / p[10]
    p[15] = 2.0 * FT / (p[0] + 0.5 * p[1]) ** 2 / p[9]
    p[16] = C[11, k]

    print(f"ATCTS was called for {Atom} with isotope index {n}.")
    return p

def THJ(j_1, j_2, j_3, m_1, m_2, m_3):
    return float(wigner_3j(j_1, j_2, j_3, m_1, m_2, m_3))



NameError: name 'p' is not defined